# 1. Import Libraries and Set Up Environment




---




In [ ]:
# !pip install datasets sacrebleu torch transformers seqeval
# !pip install sentencepiece transformers[sentencepiece]
# !pip install accelerate -U
# !pip install arabert
# !pip install rouge_score


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.1

In [ ]:
# general
import os
import random
import warnings
from typing import List,Tuple
from google.colab import drive
import sys
import ast
import re
import string
import gc
from collections import Counter
from google.colab import files
from sklearn.metrics import confusion_matrix
import argparse
import openpyxl

# ML
import numpy as np
import scipy as sp
import pandas as pd
# from datasets import load_dataset, Dataset, DatasetDict, load_metric
from sklearn.model_selection import KFold

# visual
import matplotlib
import seaborn as sns
from tqdm import tqdm
from tabulate import tabulate
import matplotlib.pyplot as plt
from IPython.display import display


# # DL
# import random
# import torch
# import torch.nn as nn
# from torch.nn.utils.rnn import pad_sequence
# from torch.utils.data import DataLoader
# from sklearn.model_selection import train_test_split

# #HF
# from transformers import AutoModel, AutoTokenizer, AutoModelForSeq2SeqLM, trainer, TrainingArguments
# from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
# from transformers import Seq2SeqTrainer

# from transformers import AutoModelForSequenceClassification, Trainer
# from transformers import DataCollatorForTokenClassification
# from torch.utils.data import Subset



In [ ]:
drive.mount('/content/drive')


Mounted at /content/drive




> Set random SEEDs



In [ ]:
# Set a random seed for NumPy (for CPU)
np.random.seed(101)

# Set a random seed for PyTorch (for GPU)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(101)

# Set a random seed for the random module
random.seed(42)

NameError: name 'torch' is not defined

# 2. Load and Prepare Dataset

---




## Prepare Dataset

In [ ]:
def read_excel_files(directory, number_limit):
    # Initialize an empty DataFrame with the desired columns
    combined_df = pd.DataFrame(columns=['speaker_original', 'text_original', 'speaker_enhanced', 'text_enhanced', 'source'])

    # List all files in the directory
    for filename in os.listdir(directory):
        # Use a regular expression to extract the number from the filename
        match = re.search(r'biqawi_#(\d+)', filename)
        if match:
            file_number = int(match.group(1))
            # Check if the file number is within the limit
            if file_number <= number_limit:
                # Full path to the file
                file_path = os.path.join(directory, filename)

                # Read the Excel file into a DataFrame, skipping the first row
                df = pd.read_excel(file_path, skiprows=1)

                # Ensure the DataFrame has the correct columns
                if df.shape[1] == 4:  # Check if there are 4 columns
                    df.columns = ['speaker_original', 'text_original', 'speaker_enhanced', 'text_enhanced']

                    # Add a new column to store the source file name
                    df['source'] = filename

                    # Append the DataFrame to the combined DataFrame
                    combined_df = pd.concat([combined_df, df], ignore_index=True)
                else:
                    print(f"Skipping file {filename} due to column mismatch")

    return combined_df

# Directory containing the Excel files in Google Drive
directory = "/content/drive/MyDrive/biqawi_alignment_model/Biqawi_alignment"

# Number limit for files to include
number_limit = 60

# Read the files and get the combined DataFrame
combined_df = read_excel_files(directory, number_limit)

Skipping file biqawi_#4.xlsx due to column mismatch


In [ ]:
unique_sources_count = combined_df['source'].nunique()
print(f"Number of unique sources: {unique_sources_count}")


Number of unique sources: 53


In [ ]:
unique_sources = combined_df['source'].unique()
print(f"Unique sources: {unique_sources}")


Unique sources: ['biqawi_#3.xlsx' 'biqawi_#2.xlsx' 'biqawi_#5.xlsx' 'biqawi_#7.xlsx'
 'biqawi_#8.xlsx' 'biqawi_#6.xlsx' 'biqawi_#12.xlsx' 'biqawi_#9.xlsx'
 'biqawi_#13.xlsx' 'biqawi_#11.xlsx' 'biqawi_#10.xlsx' 'biqawi_#14.xlsx'
 'biqawi_#17.xlsx' 'biqawi_#15.xlsx' 'biqawi_#16.xlsx' 'biqawi_#18.xlsx'
 'biqawi_#19.xlsx' 'biqawi_#23.xlsx' 'biqawi_#22.xlsx' 'biqawi_#20.xlsx'
 'biqawi_#24.xlsx' 'biqawi_#25.xlsx' 'biqawi_#26.xlsx' 'biqawi_#31.xlsx'
 'biqawi_#29.xlsx' 'biqawi_#32.xlsx' 'biqawi_#27.xlsx' 'biqawi_#28.xlsx'
 'biqawi_#36.xlsx' 'biqawi_#33.xlsx' 'biqawi_#38.xlsx' 'biqawi_#37.xlsx'
 'biqawi_#39.xlsx' 'biqawi_#34.xlsx' 'biqawi_#35.xlsx' 'biqawi_#43.xlsx'
 'biqawi_#41.xlsx' 'biqawi_#44.xlsx' 'biqawi_#46.xlsx' 'biqawi_#42.xlsx'
 'biqawi_#45.xlsx' 'biqawi_#49.xlsx' 'biqawi_#52.xlsx' 'biqawi_#48.xlsx'
 'biqawi_#50.xlsx' 'biqawi_#51.xlsx' 'biqawi_#47.xlsx' 'biqawi_#54.xlsx'
 'biqawi_#55.xlsx' 'biqawi_#53.xlsx' 'biqawi_#56.xlsx' 'biqawi_#57.xlsx'
 'biqawi_#58.xlsx']


In [ ]:
combined_df

,speaker_original,text_original,speaker_enhanced,text_enhanced,source
0,Speaker #1,المواطن السوري ال آه ال الأصيل من سكان البريد و,Speaker #1,المواطن السوري ال آه ال الأصيل من سكان البريد و,biqawi_#3.xlsx
1,Speaker #1,السوري النازح إللي إجا من الجولان والفلسطيني ...,Speaker #1,السوري النازح إللي إجا من الجولان والفلسطيني ...,biqawi_#3.xlsx
2,Speaker #1,تبعت 67، يعني أنا الآن إذا بدي أتصور لك الخريطة,Speaker #1,تبعت ال 67 يعني أنا الآن إذا بدي أصورلك الخريطة,biqawi_#3.xlsx
3,Speaker #1,تبع بيتنا على يسارنا، كان في واحد من غزة وعلى,Speaker #1,تبع بيتنا على يسارنا كان في واحد من غزة وعلى,biqawi_#3.xlsx
4,Speaker #1,يميننا كان جارنا من آه السوريين 67 الجولان,Speaker #1,يميننا كان جارنا من آه السوريين ال 67 من الجولان,biqawi_#3.xlsx
...,...,...,...,...,...
69763,Speaker #2,مشاعرو كثير تفاصيل، وأظن يعني من الحلقات القلي...,Speaker #2,تفاصيل وأظن يعني من الحلقات القليلة إللي فيها...,biqawi_#58.xlsx
69764,Speaker #2,المساحتين، جغرافيات راح يكون،,Speaker #2,مساحتين جغرافيات راح يكون.,biqawi_#58.xlsx
69765,Speaker #2,ومعني فيها آه السوريين والفلسطينيين رحبوا، يح...,Speaker #2,معني فيها السوريين والفلسطينيين رح يحبوا يحضر...,biqawi_#58.xlsx
69766,Speaker #2,فا ساهم هو ساعدونا في. في المشاركة والتواصل إ...,Speaker #2,فساهموا وساعدونا في المشاركة والتواصل لإلها و...,biqawi_#58.xlsx


### check empty values

In [ ]:
# # Fixing empty values in 'text_original' using values from 'text_enhanced'
# # If 'text_original' is NaN and 'text_enhanced' is not NaN, use the value from 'text_enhanced'
# combined_df['text_original'] = combined_df.apply(
#     lambda row: row['text_enhanced'] if pd.isna(row['text_original']) and not pd.isna(row['text_enhanced']) else row['text_original'],
#     axis=1
# )

# # Updating 'speaker_original' based on 'speaker_enhanced' if 'text_enhanced' is available
# # If 'speaker_original' is NaN and 'text_enhanced' is not NaN, use the value from 'speaker_enhanced'
# combined_df['speaker_original'] = combined_df.apply(
#     lambda row: row['speaker_enhanced'] if pd.isna(row['speaker_original']) and not pd.isna(row['text_enhanced']) else row['speaker_original'],
#     axis=1
# )


# Drop rows where either 'text_original' is NaN and 'text_enhanced' is not, or 'text_enhanced' is NaN and 'text_original' is not
combined_df = combined_df.dropna(subset=['text_original', 'text_enhanced'], how='any').reset_index(drop=True)


In [ ]:
# Filter rows with NaN values in 'text_original' or 'text_enhanced'
nan_rows = combined_df[combined_df['text_original'].isna() | combined_df['text_enhanced'].isna()]

nan_rows

,speaker_original,text_original,speaker_enhanced,text_enhanced,source


In [ ]:
#should be empty !
unique_sources = nan_rows['source'].unique()
print(f"Unique sources: {unique_sources}")


NameError: name 'nan_rows' is not defined

In [ ]:
combined_df = combined_df[combined_df['speaker_original'] != "Speaker #2"]
combined_df

,speaker_original,text_original,speaker_enhanced,text_enhanced,source
0,Speaker #1,المواطن السوري ال آه ال الأصيل من سكان البريد و,Speaker #1,المواطن السوري ال آه ال الأصيل من سكان البريد و,biqawi_#3.xlsx
1,Speaker #1,السوري النازح إللي إجا من الجولان والفلسطيني ...,Speaker #1,السوري النازح إللي إجا من الجولان والفلسطيني ...,biqawi_#3.xlsx
2,Speaker #1,تبعت 67، يعني أنا الآن إذا بدي أتصور لك الخريطة,Speaker #1,تبعت ال 67 يعني أنا الآن إذا بدي أصورلك الخريطة,biqawi_#3.xlsx
3,Speaker #1,تبع بيتنا على يسارنا، كان في واحد من غزة وعلى,Speaker #1,تبع بيتنا على يسارنا كان في واحد من غزة وعلى,biqawi_#3.xlsx
4,Speaker #1,يميننا كان جارنا من آه السوريين 67 الجولان,Speaker #1,يميننا كان جارنا من آه السوريين ال 67 من الجولان,biqawi_#3.xlsx
...,...,...,...,...,...
69750,Speaker #1,جغرافية إللي.,Speaker #1,بدمشق بالشام في تقسيمات يعني في تقسيمات جغرا...,biqawi_#58.xlsx
69752,Speaker #1,وتحاربها.,Speaker #1,طبعا يعني.,biqawi_#58.xlsx
69753,Speaker #1,بيختلف التسميات شوي مع شوية حصتهم زيادة؟ مع ش...,Speaker #1,وبتختلف التسميات شوي مع شوية حصتم زيادة مع شو...,biqawi_#58.xlsx
69754,Speaker #1,هاي زيادة؟ فهلأ؟ هاي أنا ما حبيت ها الشغلة ال...,Speaker #1,هذا زيادة ، فلا هذه أنا ما حبيتهاش شغلة اللحمة,biqawi_#58.xlsx


### assign target and source data

In [ ]:
sentences_raw = combined_df['text_original']
sentences_tuned= combined_df['text_enhanced']
print(len(sentences_raw) == len(sentences_tuned))


# Check for non-string values
non_string_raw = sentences_raw.apply(lambda x: not isinstance(x, str))
non_string_tuned = sentences_tuned.apply(lambda x: not isinstance(x, str))
print("Non-string values in 'text_original':")
print(combined_df[non_string_raw])

print("Non-string values in 'text_enhanced':")
print(combined_df[non_string_tuned])

True
Non-string values in 'text_original':
      speaker_original text_original speaker_enhanced  \
9329        Speaker #1            32       Speaker #1   
10414       Speaker #1             1       Speaker #1   
11173       Speaker #1          2012       Speaker #1   
12322       Speaker #1            20       Speaker #1   
12328       Speaker #1            25       Speaker #1   
13308       Speaker #1          2009       Speaker #1   
13337       Speaker #1          2018       Speaker #1   
15739       Speaker #1             5       Speaker #1   
16078       Speaker #1             1       Speaker #1   
16649       Speaker #1          0.05       Speaker #1   
23031       Speaker #1          2018       Speaker #1   
23806       Speaker #1             4       Speaker #1   
23906       Speaker #1           101       Speaker #1   
24115       Speaker #1          2007       Speaker #1   
24345       Speaker #1           456       Speaker #1   
31803       Speaker #1            84       Sp

In [ ]:
#convert to data frame :
data = {'source': sentences_raw, 'target': sentences_tuned, 'podcast' : combined_df['source']}
dataset  = pd.DataFrame(data)

#convert non string values to string :
# Ensure all entries in 'source' and 'target' are strings
dataset['source'] = dataset['source'].astype(str)
dataset['target'] = dataset['target'].astype(str)

In [ ]:
dataset.to_csv("/content/drive/MyDrive/biqawi_alignment_model/dataset_for_training.csv", index=False)

In [ ]:
dataset

,source,target,podcast
0,المواطن السوري ال آه ال الأصيل من سكان البريد و,المواطن السوري ال آه ال الأصيل من سكان البريد و,biqawi_#3.xlsx
1,السوري النازح إللي إجا من الجولان والفلسطيني ...,السوري النازح إللي إجا من الجولان والفلسطيني ...,biqawi_#3.xlsx
2,تبعت 67، يعني أنا الآن إذا بدي أتصور لك الخريطة,تبعت ال 67 يعني أنا الآن إذا بدي أصورلك الخريطة,biqawi_#3.xlsx
3,تبع بيتنا على يسارنا، كان في واحد من غزة وعلى,تبع بيتنا على يسارنا كان في واحد من غزة وعلى,biqawi_#3.xlsx
4,يميننا كان جارنا من آه السوريين 67 الجولان,يميننا كان جارنا من آه السوريين ال 67 من الجولان,biqawi_#3.xlsx
...,...,...,...
69761,مشاعرو كثير تفاصيل، وأظن يعني من الحلقات القلي...,تفاصيل وأظن يعني من الحلقات القليلة إللي فيها...,biqawi_#58.xlsx
69762,المساحتين، جغرافيات راح يكون،,مساحتين جغرافيات راح يكون.,biqawi_#58.xlsx
69763,ومعني فيها آه السوريين والفلسطينيين رحبوا، يح...,معني فيها السوريين والفلسطينيين رح يحبوا يحضر...,biqawi_#58.xlsx
69764,فا ساهم هو ساعدونا في. في المشاركة والتواصل إ...,فساهموا وساعدونا في المشاركة والتواصل لإلها و...,biqawi_#58.xlsx


##  Load data for training

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/biqawi_alignment_model/dataset_for_training.csv")

In [ ]:
dataset

,source,target,podcast
0,المواطن السوري ال آه ال الأصيل من سكان البريد و,المواطن السوري ال آه ال الأصيل من سكان البريد و,biqawi_#3.xlsx
1,السوري النازح إللي إجا من الجولان والفلسطيني ...,السوري النازح إللي إجا من الجولان والفلسطيني ...,biqawi_#3.xlsx
2,تبعت 67، يعني أنا الآن إذا بدي أتصور لك الخريطة,تبعت ال 67 يعني أنا الآن إذا بدي أصورلك الخريطة,biqawi_#3.xlsx
3,تبع بيتنا على يسارنا، كان في واحد من غزة وعلى,تبع بيتنا على يسارنا كان في واحد من غزة وعلى,biqawi_#3.xlsx
4,يميننا كان جارنا من آه السوريين 67 الجولان,يميننا كان جارنا من آه السوريين ال 67 من الجولان,biqawi_#3.xlsx
...,...,...,...
69761,مشاعرو كثير تفاصيل، وأظن يعني من الحلقات القلي...,تفاصيل وأظن يعني من الحلقات القليلة إللي فيها...,biqawi_#58.xlsx
69762,المساحتين، جغرافيات راح يكون،,مساحتين جغرافيات راح يكون.,biqawi_#58.xlsx
69763,ومعني فيها آه السوريين والفلسطينيين رحبوا، يح...,معني فيها السوريين والفلسطينيين رح يحبوا يحضر...,biqawi_#58.xlsx
69764,فا ساهم هو ساعدونا في. في المشاركة والتواصل إ...,فساهموا وساعدونا في المشاركة والتواصل لإلها و...,biqawi_#58.xlsx


In [ ]:
#change the order of the sentences in the tuned list:
# Function to reverse the word order of a sentence
def reverse_sentence(sentence):
    words = str(sentence).split()  # Ensure the sentence is converted to string
    reversed_words = words[::-1]
    return ' '.join(reversed_words)

In [ ]:
# Apply the function to each sentence in the 'sentences_tuned' list
sentences_tuned = sentences_tuned.apply(reverse_sentence)

# 3.Tokenization and Data Preparation

---





In [ ]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("moussaKam/AraBART", trust_remote_code=True)
model = AutoModelForSeq2SeqLM.from_pretrained("moussaKam/AraBART", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/557M [00:00<?, ?B/s]

In [ ]:
def remove_punctuation(text):
    # Define language-specific and regular punctuation
    additional_punctuation = "،؛؟"
    all_punctuation = string.punctuation + additional_punctuation
    # Remove punctuation using regular expressions
    text = re.sub(f'[{re.escape(all_punctuation)}]', '', text)
    # Remove any extra spaces created by removing punctuation
    text = re.sub(r'\s+', ' ', text).strip()
    return text

#remove punctuation
dataset['source'] = dataset['source'].apply(remove_punctuation)
dataset['target'] = dataset['target'].apply(remove_punctuation)


In [ ]:
#finding the max length
max_length_source = dataset['source'].str.len().max()
max_length_target = dataset['target'].str.len().max()
print(max_length_target,max_length_source)


91 97


In [ ]:
dataset

,source,target,podcast
0,المواطن السوري ال آه ال الأصيل من سكان البريد و,المواطن السوري ال آه ال الأصيل من سكان البريد و,biqawi_#3.xlsx
1,السوري النازح إللي إجا من الجولان والفلسطيني ت...,السوري النازح إللي إجا من الجولان والفلسطيني ت...,biqawi_#3.xlsx
2,تبعت 67 يعني أنا الآن إذا بدي أتصور لك الخريطة,تبعت ال 67 يعني أنا الآن إذا بدي أصورلك الخريطة,biqawi_#3.xlsx
3,تبع بيتنا على يسارنا كان في واحد من غزة وعلى,تبع بيتنا على يسارنا كان في واحد من غزة وعلى,biqawi_#3.xlsx
4,يميننا كان جارنا من آه السوريين 67 الجولان,يميننا كان جارنا من آه السوريين ال 67 من الجولان,biqawi_#3.xlsx
...,...,...,...
69761,مشاعرو كثير تفاصيل وأظن يعني من الحلقات القليل...,تفاصيل وأظن يعني من الحلقات القليلة إللي فيها ...,biqawi_#58.xlsx
69762,المساحتين جغرافيات راح يكون,مساحتين جغرافيات راح يكون,biqawi_#58.xlsx
69763,ومعني فيها آه السوريين والفلسطينيين رحبوا يحضر...,معني فيها السوريين والفلسطينيين رح يحبوا يحضرو...,biqawi_#58.xlsx
69764,فا ساهم هو ساعدونا في في المشاركة والتواصل إله...,فساهموا وساعدونا في المشاركة والتواصل لإلها وي...,biqawi_#58.xlsx


In [ ]:
def postprocess_text(preds: list, labels: list) -> tuple:
    """Performs post processing on the prediction text and labels"""

    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def prep_data_for_model_fine_tuning(source, target):
    """
    Takes the input data lists (source and target) and converts them into a list of dictionaries
    suitable for model fine-tuning.

    Parameters:
        source (list of str): A list of source texts.
        target (list of str): A list of target texts corresponding to the source texts.

    Returns:
        List[Dict[str, str]]: A list of dictionaries, where each dictionary contains 'source' and 'target' keys.
    """
    # Check if the source and target lists have the same length
    if len(source) != len(target):
        raise ValueError("Source and target lists must have the same number of elements")

    # Create a list of dictionaries for model fine-tuning
    data_for_fine_tuning = [{'source': src, 'target': tgt} for src, tgt in zip(source, target)]

    return data_for_fine_tuning


from transformers import AutoTokenizer

def generate_model_ready_dataset(dataset: list,
                                 model_checkpoint: str,
                                 tokenizer: AutoTokenizer,
                                 max_input_length,
                                 max_target_length):
    """
    Makes the data training ready for the model based on the modified assumptions.

    Parameters:
        dataset (list): A list of dictionaries with 'source' and 'target' texts for translation.
        model_checkpoint (str): The path or identifier of the model checkpoint.
        tokenizer (AutoTokenizer): The tokenizer initialized with the model checkpoint.
        max_input_length (int): The maximum length of the tokenized input sequences.
        max_target_length (int): The maximum length of the tokenized target sequences.

    Returns:
        List[Dict]: A list of dictionaries, each containing tokenized model inputs and labels.
    """

    prepped_data = []

    for row in dataset:
        # Tokenize the source texts
        model_inputs = tokenizer(row['source'], max_length=max_input_length, truncation=True, padding='max_length')

        # Tokenize the target texts - No need for 'with tokenizer.as_target_tokenizer()' as labels are handled directly
        labels = tokenizer(row['target'], max_length=max_target_length, truncation=True, padding='max_length')

        # Assign 'labels' from the tokenized targets for model inputs
        model_inputs['labels'] = labels['input_ids']
        prepped_data.append(model_inputs)

    return prepped_data



def compute_metrics(eval_preds):
    import numpy as np
    from datasets import load_metric

    bleu_metric = load_metric("sacrebleu", trust_remote_code=True)
    rouge_metric = load_metric("rouge")
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, trust_remote_code=True)

    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    # Ensure token IDs are within valid range
    preds = np.clip(preds, 0, tokenizer.vocab_size - 1)
    labels = np.clip(labels, 0, tokenizer.vocab_size - 1)

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    post_processed_preds, post_processed_labels = postprocess_text(decoded_preds, decoded_labels)

    wrapped_references = [[label] for label in post_processed_labels]
    bleu_result = bleu_metric.compute(predictions=post_processed_preds, references=wrapped_references)
    bleu_score = {"bleu_score": bleu_result["score"]}

    rouge_result = rouge_metric.compute(predictions=post_processed_preds, references=wrapped_references)
    rouge_score = {key: value.mid.fmeasure * 100 for key, value in rouge_result.items()}

    result = {**bleu_score, **rouge_score}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}

    return result


In [ ]:
# #train test and validation:

# x = dataset['source'].copy()
# y = dataset['target'].copy()

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10,
#                                                     shuffle=True,
#                                                     random_state=100)

# print("INITIAL X-TRAIN SHAPE: ", x_train.shape)
# print("INITIAL Y-TRAIN SHAPE: ", y_train.shape)
# print("X-TEST SHAPE: ", x_test.shape)
# print("Y-TEST SHAPE: ", y_test.shape)

# x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,
#                                                   test_size=0.20,
#                                                   shuffle=True,
#                                                   random_state=100)

# print("FINAL X-TRAIN SHAPE: ", x_train.shape)
# print("FINAL Y-TRAIN SHAPE: ", y_train.shape)
# print("X-VAL SHAPE: ", x_val.shape)
# print("Y-VAL SHAPE: ", y_val.shape)


INITIAL X-TRAIN SHAPE:  (54693,)
INITIAL Y-TRAIN SHAPE:  (54693,)
X-TEST SHAPE:  (6078,)
Y-TEST SHAPE:  (6078,)


In [ ]:
#shuffle keeping podcast data in the same category
# Assuming 'dataset' is your DataFrame
# Ensure 'podcast' column is treated as categorical
dataset['podcast'] = dataset['podcast'].astype('category')

# Get unique podcasts
unique_podcasts = dataset['podcast'].unique()

# Shuffle the podcasts
shuffled_podcasts = list(unique_podcasts)
random.shuffle(shuffled_podcasts)

# Split the podcasts into train, validation, and test sets
test_podcasts = shuffled_podcasts[:6]
val_podcasts = shuffled_podcasts[6:16]
train_podcasts = shuffled_podcasts[16:]

# Create the train, validation, and test sets
train_set = dataset[dataset['podcast'].isin(train_podcasts)]
val_set = dataset[dataset['podcast'].isin(val_podcasts)]
test_set = dataset[dataset['podcast'].isin(test_podcasts)]

# Separate features and target
x_train = train_set['source']
y_train = train_set['target']

x_val = val_set['source']
y_val = val_set['target']

x_test = test_set['source']
y_test = test_set['target']

# Modeling

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("moussaKam/AraBART")
model_checkpoint  = "moussaKam/AraBART"

max_input_length = 128  # Adjust as needed
max_target_length = 128 # Adjust as needed

In [ ]:
training_data = prep_data_for_model_fine_tuning(x_train.values, y_train.values)

validation_data = prep_data_for_model_fine_tuning(x_val.values, y_val.values)

test_data = prep_data_for_model_fine_tuning(x_test.values, y_test.values)

In [ ]:
# Prepare the training data
train_data = generate_model_ready_dataset(
    dataset=training_data,
    model_checkpoint=model_checkpoint,
    tokenizer=tokenizer,
    max_input_length=max_input_length,
    max_target_length=max_target_length
)

# Prepare the validation data
validation_data = generate_model_ready_dataset(
    dataset=validation_data,
    model_checkpoint=model_checkpoint,
    tokenizer=tokenizer,
    max_input_length=max_input_length,
    max_target_length=max_target_length
)

# Prepare the test data
test_data = generate_model_ready_dataset(
    dataset=test_data,
    model_checkpoint=model_checkpoint,
    tokenizer=tokenizer,
    max_input_length=max_input_length,
    max_target_length=max_target_length
)


In [ ]:
train_df = pd.DataFrame.from_records(train_data)
train_df.info()

validation_df = pd.DataFrame.from_records(validation_data)
validation_df.info()

test_df = pd.DataFrame.from_records(test_data)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47966 entries, 0 to 47965
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   attention_mask  47966 non-null  object
 1   input_ids       47966 non-null  object
 2   labels          47966 non-null  object
dtypes: object(3)
memory usage: 1.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13789 entries, 0 to 13788
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   attention_mask  13789 non-null  object
 1   input_ids       13789 non-null  object
 2   labels          13789 non-null  object
dtypes: object(3)
memory usage: 323.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8011 entries, 0 to 8010
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   attention_mask  8011 non-null   object
 1   input_ids       801

In [ ]:
#Convert dataframe to Dataset Class object
train_dataset = Dataset.from_pandas(train_df)


validation_dataset = Dataset.from_pandas(validation_df)


test_dataset = Dataset.from_pandas(test_df)

In [ ]:
from transformers import AutoConfig
# Load the configuration
config = AutoConfig.from_pretrained(model_checkpoint)

# # Print the entire configuration
# print(config)

# # Access specific attributes
# print("Number of attention heads:", config.num_attention_heads)
# print("Hidden size:", config.hidden_size)
# print("Model type:", config.model_type)




In [ ]:
from transformers import GenerationConfig, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

# Define the generation configuration
generation_config = GenerationConfig(
    max_length=128,
    num_beams=4,
    no_repeat_ngram_size=3,
    early_stopping=True,
    forced_eos_token_id=2
)

# Save the generation configuration
generation_config.save_pretrained("generation_config")

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="Arabic-finetuned",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.04,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    generation_max_length=generation_config.max_length  # This should be set in GenerationConfig
)

# Define the data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import gc
gc.collect()


30

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu Score,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.082100,0.086273,43.080100,2.653200,0.396000,2.652600,2.651700,19.008000
2,0.059000,0.081633,43.329400,2.688300,0.455900,2.683900,2.682300,19.059000
3,0.042200,0.084038,43.357500,2.686600,0.442300,2.684800,2.680100,18.046600


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generatio

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generatio

TrainOutput(global_step=8994, training_loss=0.07937199480405828, metrics={'train_runtime': 8031.5032, 'train_samples_per_second': 17.917, 'train_steps_per_second': 1.12, 'total_flos': 1.0967821606453248e+16, 'train_loss': 0.07937199480405828, 'epoch': 3.0})

In [ ]:
# print(model)

In [ ]:
# print(model_args)

In [ ]:
# print(data_collator)

In [ ]:
model.save_pretrained("/content/drive/MyDrive/biqawi_alignment_model/model")
tokenizer.save_pretrained("/content/drive/MyDrive/biqawi_alignment_model/model")


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}


('/content/drive/MyDrive/biqawi_alignment_model/model/tokenizer_config.json',
 '/content/drive/MyDrive/biqawi_alignment_model/model/special_tokens_map.json',
 '/content/drive/MyDrive/biqawi_alignment_model/model/sentencepiece.bpe.model',
 '/content/drive/MyDrive/biqawi_alignment_model/model/added_tokens.json',
 '/content/drive/MyDrive/biqawi_alignment_model/model/tokenizer.json')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}


('/content/drive/MyDrive/biqawi_alignment_model/model/tokenizer_config.json',
 '/content/drive/MyDrive/biqawi_alignment_model/model/special_tokens_map.json',
 '/content/drive/MyDrive/biqawi_alignment_model/model/sentencepiece.bpe.model',
 '/content/drive/MyDrive/biqawi_alignment_model/model/added_tokens.json',
 '/content/drive/MyDrive/biqawi_alignment_model/model/tokenizer.json')

In [ ]:
model.save_pretrained("/content/drive/MyDrive/biqawi_alignment_model/reversed_model")
tokenizer.save_pretrained("/content/drive/MyDrive/biqawi_alignment_model/reversed_model")

NameError: name 'model' is not defined

### Tests

#### straight model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_path = "/content/drive/MyDrive/biqawi_alignment_model/model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [ ]:
texts = [str(text) for text in x_test]
texts

['لكن أنا ك حسام شقلت أكثر إنه عن جد على',
 'جماعة نحنا في غزة أنا أنا بحزن على الناس إنه',
 'ما في حدا مدور علينا',
 'أنا ما أصلا حدا من الأشخاص بيقولوا إنه نحنا بنبعت',
 'تبرعات لغزة بشكل كبير',
 'قلنا له حلوو شيء كويس وغزة بيحبوك وبيدعو لكوا قال',
 'نحنا بنبعت لهم عشان نشرب من التهرب الضريبي في الدولة تبعتنا',
 'تخيل لوين يعني هو أنا كثير كثير ناس بتيجي',
 'بتقول لي غزة باكستان فلسطين باكستان يا عمي غزة فلسطين',
 'إسرائيل لا لا باكستان ففي في حتى هذا الشيء موجود',
 'عند ناس مثلا الناس عرب عندهم نقص دم وهل أنتم عن',
 'جد في غزة مش قادرين تسافروا طب أنتم عندكم حصار',
 'وما عندكم إنترنت يا جماعة نحنا عايشين والله العظيم عايشين',
 'ومبسوطين وعنا بحر وعنا ناس بتطلب نقص في ناس بتغني وناس',
 'بتدبك وعايشين حياتنا عنا الحصار إنه نحنا مسكرة على عقولنا',
 'أكثر ا ب',
 'مرحبا يا حسام',
 'أهلا يا أحمد كيفك',
 'صباح الخير يا حسام هاهاها',
 'الله أمور الدنيا رمضان',
 'صحيناك برمضان يا حسام هاهاها',
 'لا صاحي بدري خلاص معي',
 'صح هذا أروح أجرب 17 يوم بس بنحاول يعني',
 'نستغل النهار في شيء كويس',
 

In [ ]:
true_translation = [str(text) for text in y_test]
true_translation

['لكن أنا كحسام شقنت أكثر إنه عن جد على',
 'جماعة نحنا في غزة أنا بحزن على الناس إنه',
 'ما في حدا مدور علينا',
 'أنا ما أصلا حدا من الأشخاص بيقولوا إنه نحنا بنبعت',
 'تبرعات لغزة بشكل كبير',
 'قلناله حلو وشيء كويس وغزة بيحبوكوا بيدعولكوا قال',
 'نحنا بنبعث لهم عشان نشرد من التهرب الضريبه في الدولة تبعتنا',
 'تخيل لوين يعني هو أنا غلط كثير كثير ناس بتيجي',
 'بتقولي غزة باكستان فلسطين باكستان يا عمي غزة فلسطين',
 'إسرائيل لا لا باكستان ففي في حتى هذا الشيء موجود',
 'عند ناس مثلا الناس عرب إنه بنصدمو هل أنتم عن',
 'جد في غزة مش قادرين تسافروا طيب أنتم عندكم حصار',
 'وما عندكم إنترنت يا جماعة نحنا عايشين والله العظيم عايشين',
 'ومبسوطين وعنا بحر وعنا ناس بتطلب بترقص الناس بتغني وناس',
 'بتدبك وعايشين حياتنا عنا الحصار إنه نحنا مسكرة على عقولنا',
 'أكثر',
 'مرحبا يا حسام',
 'أهلا يا أحمد كيفك',
 'صباح الخير يا حسام',
 'صباح النور الدنيا رمضان',
 'صحيناك برمضان يا حسام',
 'لا صاحي بدري خلاص معيد',
 'صح هذا أروح أجر ب 17 يوم بس بنحاول يعني',
 'نستغل النهار في شيء كويس',
 'وبايش بدك تستغله أص

In [ ]:
# Initialize lists to hold original texts and their translations
original_texts = []
translated_texts = []

# Set the batch size
batch_size = 16  # Adjust this based on your available memory

# Process the texts in batches
for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i + batch_size]

    # Tokenize the input texts
    inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=256)

    # Generate translations with max_new_tokens set to 128
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=128)

    # Decode the translations
    translations = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    # Append the batch results to the lists
    original_texts.extend(batch_texts)
    translated_texts.extend(translations)

# Save the original texts and translations in a DataFrame
pred_df = pd.DataFrame({'Original': original_texts, 'Translation': translated_texts})

NameError: name 'tokenizer' is not defined

In [ ]:
# Rename and drop irrelevant columns in og_df
true_translation = [str(text) for text in y_test]
og_df = pd.DataFrame({'Original': texts, 'TrueTranslation': true_translation})
og_df['TrueTranslation'] = og_df['TrueTranslation'].apply(remove_punctuation)
og_df

,Original,TrueTranslation
0,لكن أنا ك حسام شقلت أكثر إنه عن جد على,لكن أنا كحسام شقنت أكثر إنه عن جد على
1,جماعة نحنا في غزة أنا أنا بحزن على الناس إنه,جماعة نحنا في غزة أنا بحزن على الناس إنه
2,ما في حدا مدور علينا,ما في حدا مدور علينا
3,أنا ما أصلا حدا من الأشخاص بيقولوا إنه نحنا بنبعت,أنا ما أصلا حدا من الأشخاص بيقولوا إنه نحنا بنبعت
4,تبرعات لغزة بشكل كبير,تبرعات لغزة بشكل كبير
...,...,...
7394,مشاعرو كثير تفاصيل وأظن يعني من الحلقات القليل...,تفاصيل وأظن يعني من الحلقات القليلة إللي فيها ...
7395,المساحتين جغرافيات راح يكون,مساحتين جغرافيات راح يكون
7396,ومعني فيها آه السوريين والفلسطينيين رحبوا يحضر...,معني فيها السوريين والفلسطينيين رح يحبوا يحضرو...
7397,فا ساهم هو ساعدونا في في المشاركة والتواصل إله...,فساهموا وساعدونا في المشاركة والتواصل لإلها وي...


In [ ]:
pred_df


,Original,Translation
0,كمان بس من ناحية مضمونية أنا دائما أنا إنسان إللي,كمان بس من ناحية مضمونية أنا دائما أنا إنسان إللي
1,هو دائما بالتواصل مع الطفل إللي بداخله أنا,هو دائما بالتواصل مع الطفل إللي بداخله أنا
2,ما ما ما بخليهم شيء كبار مابخليها الشيء كبار ف...,ما بخليهوش يكبر في طفولتنا أعتقد
3,إحنا كلنا بالأخص نحنا كعرب وبالأخص كذكور إللي ...,إحنا كلنا بالأخص نحنا كعرب وبالأخص كذكور إللي ...
4,كان ما كان له وسع ما كان لإله وما,كان ما كان إله وسع ما كان لإله وسع وما
...,...,...
8006,لإلكم وشكرا على المتابعة إيه وبحب أذكرك معبد أ...,لإلكم وشكرا على المتابعة وبحب أذكرك معبد أشطر مني
8007,بالتذكير أنا بضل أنسى بالتذكير بالمشاركة المشا...,بالتذكير أنا بضل أنسى بالتذكير بالمشاركة المشا...
8008,بتعطي طاقة وبتعطي يعني طاقة عن جد دفعة كبيرة ل...,بتعطي طاقة وبتعطي يعني طاقة عن جد دفعة كبيرة ل...
8009,أكثر وللبحث على ضيوف وعلى مواضيع بتهم بتهمكم ش...,أكثر والبحث على ضيوف وعلى مواضيع بتهم بتهمكم ش...


In [ ]:
'''
This script performs several tasks related to translation evaluation and comparison.
1. **Data Merging**:
   - It merges two DataFrames (`og_df` and `df`) based on the common column 'Original'.
   - The resulting DataFrame is stored in `merged_df`.

2. **BLEU Score Calculation**:
   - The `calculate_bleu` function computes the BLEU score between a reference translation and a candidate translation.
   - Both the reference and candidate translations are tokenized using `.split()`.
   - The BLEU score is calculated with smoothing using `SmoothingFunction().method1`.
   - The scores are added to the `merged_df` DataFrame.

3. **Exact Match Check**:
   - An 'Exact_Match' column is added to `merged_df`, indicating whether the candidate translation exactly matches the true reference translation.

4. **Tokenization for Comparison**:
   - The true and model translations are tokenized into lists of words (`TrueTokens` and `ModelTokens`) for further comparison.

5. **Identifying Mismatches**:
   - The `collect_mismatches` function compares corresponding tokens in the true and model translations.
   - If a token mismatch occurs, it's recorded in the 'Mismatches' column of `merged_df`.

6. **Counting and Displaying Common Mistakes**:
   - All mismatches are flattened into a single list (`all_mismatches`).
   - The `Counter` is used to count occurrences of each mismatch.
   - The most common mistakes are printed to the console.

7. **Optional: Saving Mistakes to CSV**:
   - If desired, the common mistakes can be saved to a CSV file in a DataFrame called `mistakes_df`.
'''
# Import necessary libraries
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from collections import Counter

merged_df = pd.DataFrame({
    'Original': texts,
    'TrueTranslation': og_df['TrueTranslation'],
    'Translation': pred_df['Translation']
})

# Function to calculate BLEU score for each pair of translations
def calculate_bleu(reference, candidate):
    reference = reference.split()  # Tokenize the reference translation
    candidate = candidate.split()  # Tokenize the candidate translation
    smoothing_function = SmoothingFunction().method1
    return sentence_bleu([reference], candidate, smoothing_function=smoothing_function)

# Calculate BLEU scores and exact matches
merged_df['BLEU_Score'] = merged_df.apply(lambda row: calculate_bleu(row['TrueTranslation'], row['Translation']), axis=1)
merged_df['Exact_Match'] = merged_df['TrueTranslation'] == merged_df['Translation']

# Tokenize the translations for comparison
merged_df['TrueTokens'] = merged_df['TrueTranslation'].apply(lambda x: x.split())  # Tokenize true translations
merged_df['ModelTokens'] = merged_df['Translation'].apply(lambda x: x.split())  # Tokenize model translations

# Identify and collect mismatches
def collect_mismatches(true_tokens, model_tokens):
    mismatches = []
    for true, model in zip(true_tokens, model_tokens):
        if true != model:
            mismatches.append((true, model))
    return mismatches

merged_df['Mismatches'] = merged_df.apply(lambda row: collect_mismatches(row['TrueTokens'], row['ModelTokens']), axis=1)

# Flatten the list of mismatches and count occurrences
all_mismatches = [mismatch for sublist in merged_df['Mismatches'] for mismatch in sublist]
mismatch_counter = Counter(all_mismatches)

# Display the most common mistakes
common_mistakes = mismatch_counter.most_common()
# for mistake, count in common_mistakes:
#     print(f"Mistake: {mistake}, Count: {count}")

# Optionally, save the common mistakes to a CSV file
mistakes_df = pd.DataFrame(common_mistakes, columns=['Mistake', 'Count'])
mistakes_df

ValueError: array length 7399 does not match index length 8011

In [ ]:
# from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
# from nltk.util import ngrams

# def calculate_bleu_alternative(reference, candidate, max_n=4):
#     # Tokenize reference and candidate
#     reference_tokens = reference.split()
#     candidate_tokens = candidate.split()

#     # Collect n-grams
#     reference_ngrams = [ngrams(reference_tokens, n) for n in range(1, max_n + 1)]
#     candidate_ngrams = [ngrams(candidate_tokens, n) for n in range(1, max_n + 1)]

#     # Initialize counts
#     total_counts = [0] * max_n
#     match_counts = [0] * max_n

#     # Count matching n-grams
#     for n in range(max_n):
#         reference_set = set(reference_ngrams[n])
#         candidate_set = set(candidate_ngrams[n])
#         total_counts[n] = len(candidate_set)
#         match_counts[n] = len(reference_set.intersection(candidate_set))

#     # Calculate precision for each n-gram
#     precisions = [match_counts[n] / (total_counts[n] + 1e-10) for n in range(max_n)]

#     # Brevity penalty
#     reference_length = len(reference_tokens)
#     candidate_length = len(candidate_tokens)
#     brevity_penalty = min(1.0, candidate_length / reference_length)

#     # Combine precisions using geometric mean
#     bleu_score = brevity_penalty * (precisions[0] * precisions[1] * precisions[2] * precisions[3]) ** 0.25

#     return bleu_score



In [ ]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.util import ngrams
from collections import Counter

# Merge the DataFrames on the 'Original' column
merged_df = pd.DataFrame({
    'Original': texts,
    'TrueTranslation': og_df['TrueTranslation'],
    'Translation': pred_df['Translation']
})

# Function to calculate BLEU score for each pair of translations
def calculate_bleu(reference, candidate):
    reference = reference.split()
    candidate = candidate.split()
    smoothing_function = SmoothingFunction().method1
    return sentence_bleu([reference], candidate, smoothing_function=smoothing_function)

# Function to calculate ROUGE scores
def calculate_rouge(reference, candidate):
    reference_tokens = reference.split()
    candidate_tokens = candidate.split()

    # Unigrams (ROUGE-1)
    reference_unigrams = set(ngrams(reference_tokens, 1))
    candidate_unigrams = set(ngrams(candidate_tokens, 1))
    common_unigrams = reference_unigrams.intersection(candidate_unigrams)
    precision_1 = len(common_unigrams) / (len(candidate_unigrams) + 1e-10)
    recall_1 = len(common_unigrams) / (len(reference_unigrams) + 1e-10)

    # Bigrams (ROUGE-2)
    reference_bigrams = set(ngrams(reference_tokens, 2))
    candidate_bigrams = set(ngrams(candidate_tokens, 2))
    common_bigrams = reference_bigrams.intersection(candidate_bigrams)
    precision_2 = len(common_bigrams) / (len(candidate_bigrams) + 1e-10)
    recall_2 = len(common_bigrams) / (len(reference_bigrams) + 1e-10)

    # F1-score for ROUGE-1 and ROUGE-2
    f1_1 = 2 * (precision_1 * recall_1) / (precision_1 + recall_1 + 1e-10)
    f1_2 = 2 * (precision_2 * recall_2) / (precision_2 + recall_2 + 1e-10)

    return round(f1_1, 4), round(f1_2, 4)

# Calculate BLEU and ROUGE scores, and exact matches
merged_df['BLEU_Score'] = merged_df.apply(lambda row: calculate_bleu(row['TrueTranslation'], row['Translation']), axis=1)
merged_df[['ROUGE-1', 'ROUGE-2']] = merged_df.apply(lambda row: calculate_rouge(row['TrueTranslation'], row['Translation']), axis=1, result_type='expand')
merged_df['Exact_Match'] = merged_df['TrueTranslation'] == merged_df['Translation']

# Calculate median and mean for BLEU_Score, ROUGE-1, and ROUGE-2
metrics_summary = {
    'BLEU_Score': {
        'mean': merged_df['BLEU_Score'].mean(),
        'median': merged_df['BLEU_Score'].median()
    },
    'ROUGE-1': {
        'mean': merged_df['ROUGE-1'].mean(),
        'median': merged_df['ROUGE-1'].median()
    },
    'ROUGE-2': {
        'mean': merged_df['ROUGE-2'].mean(),
        'median': merged_df['ROUGE-2'].median()
    }
}

# Display the updated summary and metrics summary
summary = merged_df[['Original', 'TrueTranslation', 'Translation', 'BLEU_Score', 'ROUGE-1', 'ROUGE-2', 'Exact_Match']]
summary


,Original,TrueTranslation,Translation,BLEU_Score,ROUGE-1,ROUGE-2,Exact_Match
0,كمان بس من ناحية مضمونية أنا دائما أنا إنسان إللي,كمان بس من ناحية مضمونية أنا دائما أنا إنسان إللي,كمان بس من ناحية مضمونية أنا دائما أنا إنسان إللي,1.000000,1.0,1.0000,True
1,هو دائما بالتواصل مع الطفل إللي بداخله أنا,هو دائما بالتواصل مع الطفل إللي بداخله أنا,هو دائما بالتواصل مع الطفل إللي بداخله أنا,1.000000,1.0,1.0000,True
2,ما ما ما بخليهم شيء كبار مابخليها الشيء كبار ف...,ما بخليهوش يكبر ما بخليهوش يكبر في طفولتنا أعتقد,ما بخليهوش يكبر في طفولتنا أعتقد,0.606531,1.0,0.9091,False
3,إحنا كلنا بالأخص نحنا كعرب وبالأخص كذكور إللي ...,إحنا كلنا بالأخص نحنا كعرب وبالأخص كذكور إللي ...,إحنا كلنا بالأخص نحنا كعرب وبالأخص كذكور إللي ...,1.000000,1.0,1.0000,True
4,كان ما كان له وسع ما كان لإله وما,كان ما كان إله وسع ما كان لإله وسع وما,كان ما كان إله وسع ما كان لإله وسع وما,1.000000,1.0,1.0000,True
...,...,...,...,...,...,...,...
8006,لإلكم وشكرا على المتابعة إيه وبحب أذكرك معبد أ...,لإلكم وشكرا على المتابعة وبحب أذكرك معبد أشطر مني,لإلكم وشكرا على المتابعة وبحب أذكرك معبد أشطر مني,1.000000,1.0,1.0000,True
8007,بالتذكير أنا بضل أنسى بالتذكير بالمشاركة المشا...,بالتذكير أنا بضل أنسى بالتذكير بالمشاركة المشا...,بالتذكير أنا بضل أنسى بالتذكير بالمشاركة المشا...,1.000000,1.0,1.0000,True
8008,بتعطي طاقة وبتعطي يعني طاقة عن جد دفعة كبيرة ل...,بتعطي طاقة وبتعطي يعني طاقة عن جد دفعة كبيرة ل...,بتعطي طاقة وبتعطي يعني طاقة عن جد دفعة كبيرة ل...,1.000000,1.0,1.0000,True
8009,أكثر وللبحث على ضيوف وعلى مواضيع بتهم بتهمكم ش...,أكثر والبحث على ضيوف وعلى مواضيع بتهم بتهمكم ش...,أكثر والبحث على ضيوف وعلى مواضيع بتهم بتهمكم ش...,1.000000,1.0,1.0000,True


In [ ]:
metrics_summary

{'BLEU_Score': {'mean': 0.7577717653473676, 'median': 1.0},
 'ROUGE-1': {'mean': 0.9242872300586693, 'median': 1.0},
 'ROUGE-2': {'mean': 0.8332506927974036, 'median': 1.0}}

In [ ]:
# Optionally, save the DataFrame to a CSV file
summary.to_csv("/content/drive/MyDrive/biqawi_alignment_model/translations_straight.csv", index=False)

In [ ]:
summary = pd.read_csv("/content/drive/MyDrive/biqawi_alignment_model/translations_straight.csv")

In [ ]:
filtered_summary = summary[(summary['BLEU_Score'] != 1) & (summary['Exact_Match'] == True)]
filtered_summary

,Original,TrueTranslation,Translation,BLEU_Score,ROUGE-1,ROUGE-2,Exact_Match
9,تامر مرحبا,تامر مرحبا,تامر مرحبا,0.316228,1.0,1.0,True
10,شو الوضع,شو الوضع,شو الوضع,0.316228,1.0,1.0,True
11,كيف حالك,كيف حالك,كيف حالك,0.316228,1.0,1.0,True
12,تمام الحمد لله,تمام الحمد لله,تمام الحمد لله,0.562341,1.0,1.0,True
20,وهيك,وهيك,وهيك,0.177828,1.0,0.0,True
...,...,...,...,...,...,...,...
7986,يعني من قلب,يعني من قلب,يعني من قلب,0.562341,1.0,1.0,True
7994,46,46,46,0.177828,1.0,0.0,True
7996,الخميس,الخميس,الخميس,0.177828,1.0,0.0,True
8004,آه وبالتوفيق بالبرنامج,وبالتوفيق بالبرنامج,وبالتوفيق بالبرنامج,0.316228,1.0,1.0,True


In [ ]:
summary = pd.read_csv("/content/drive/MyDrive/biqawi_alignment_model/translations_straight.csv")
a_c = summary[['Original','TrueTranslation']].copy()

In [ ]:
#Calculate a_c
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.util import ngrams

# Assuming you have the a_c DataFrame with columns 'Original' and 'Translation'

# Function to calculate BLEU score for each pair of translations
def calculate_bleu(reference, candidate):
    reference = reference.split()
    candidate = candidate.split()
    smoothing_function = SmoothingFunction().method1
    return sentence_bleu([reference], candidate, smoothing_function=smoothing_function)

# Function to calculate ROUGE scores
def calculate_rouge(reference, candidate):
    reference_tokens = reference.split()
    candidate_tokens = candidate.split()

    # Unigrams (ROUGE-1)
    reference_unigrams = set(ngrams(reference_tokens, 1))
    candidate_unigrams = set(ngrams(candidate_tokens, 1))
    common_unigrams = reference_unigrams.intersection(candidate_unigrams)
    precision_1 = len(common_unigrams) / (len(candidate_unigrams) + 1e-10)
    recall_1 = len(common_unigrams) / (len(reference_unigrams) + 1e-10)

    # Bigrams (ROUGE-2)
    reference_bigrams = set(ngrams(reference_tokens, 2))
    candidate_bigrams = set(ngrams(candidate_tokens, 2))
    common_bigrams = reference_bigrams.intersection(candidate_bigrams)
    precision_2 = len(common_bigrams) / (len(candidate_bigrams) + 1e-10)
    recall_2 = len(common_bigrams) / (len(reference_bigrams) + 1e-10)

    # F1-score for ROUGE-1 and ROUGE-2
    f1_1 = 2 * (precision_1 * recall_1) / (precision_1 + recall_1 + 1e-10)
    f1_2 = 2 * (precision_2 * recall_2) / (precision_2 + recall_2 + 1e-10)

    return round(f1_1, 4), round(f1_2, 4)

# Calculate BLEU and ROUGE scores, and exact matches for each row in a_c
a_c['BLEU_Score'] = a_c.apply(lambda row: calculate_bleu(row['Original'], row['TrueTranslation']), axis=1)
a_c[['ROUGE-1', 'ROUGE-2']] = a_c.apply(lambda row: calculate_rouge(row['Original'], row['TrueTranslation']), axis=1, result_type='expand')
a_c['Exact_Match'] = a_c['Original'] == a_c['TrueTranslation']

# Calculate median and mean for BLEU_Score, ROUGE-1, and ROUGE-2
metrics_summary = {
    'BLEU_Score': {
        'mean': a_c['BLEU_Score'].mean(),
        'median': a_c['BLEU_Score'].median()
    },
    'ROUGE-1': {
        'mean': a_c['ROUGE-1'].mean(),
        'median': a_c['ROUGE-1'].median()
    },
    'ROUGE-2': {
        'mean': a_c['ROUGE-2'].mean(),
        'median': a_c['ROUGE-2'].median()
    }
}

# Display the updated summary and metrics summary
summary = a_c[['Original', 'TrueTranslation', 'BLEU_Score', 'ROUGE-1', 'ROUGE-2', 'Exact_Match']]
summary


,Original,TrueTranslation,BLEU_Score,ROUGE-1,ROUGE-2,Exact_Match
0,كمان بس من ناحية مضمونية أنا دائما أنا إنسان إللي,كمان بس من ناحية مضمونية أنا دائما أنا إنسان إللي,1.000000,1.0000,1.0000,True
1,هو دائما بالتواصل مع الطفل إللي بداخله أنا,هو دائما بالتواصل مع الطفل إللي بداخله أنا,1.000000,1.0000,1.0000,True
2,ما ما ما بخليهم شيء كبار مابخليها الشيء كبار ف...,ما بخليهوش يكبر ما بخليهوش يكبر في طفولتنا أعتقد,0.096625,0.5333,0.2500,False
3,إحنا كلنا بالأخص نحنا كعرب وبالأخص كذكور إللي ...,إحنا كلنا بالأخص نحنا كعرب وبالأخص كذكور إللي ...,1.000000,1.0000,1.0000,True
4,كان ما كان له وسع ما كان لإله وما,كان ما كان إله وسع ما كان لإله وسع وما,0.392815,0.8333,0.5333,False
...,...,...,...,...,...,...
8006,لإلكم وشكرا على المتابعة إيه وبحب أذكرك معبد أ...,لإلكم وشكرا على المتابعة وبحب أذكرك معبد أشطر مني,0.669048,0.9474,0.8235,False
8007,بالتذكير أنا بضل أنسى بالتذكير بالمشاركة المشا...,بالتذكير أنا بضل أنسى بالتذكير بالمشاركة المشا...,1.000000,1.0000,1.0000,True
8008,بتعطي طاقة وبتعطي يعني طاقة عن جد دفعة كبيرة ل...,بتعطي طاقة وبتعطي يعني طاقة عن جد دفعة كبيرة ل...,1.000000,1.0000,1.0000,True
8009,أكثر وللبحث على ضيوف وعلى مواضيع بتهم بتهمكم ش...,أكثر والبحث على ضيوف وعلى مواضيع بتهم بتهمكم ش...,0.782542,0.9000,0.7778,False


In [ ]:
metrics_summary

{'BLEU_Score': {'mean': 0.6517606756480695, 'median': 0.6690484408935986},
 'ROUGE-1': {'mean': 0.8828272125826989, 'median': 0.9412},
 'ROUGE-2': {'mean': 0.7572131444264136, 'median': 0.8235}}

In [ ]:
len(texts)

4817

In [ ]:
# Initialize an empty list to store duplicates
duplicates = []

# Check for duplicates and remove them from the dataframe
def check_duplicates_and_remove(df, train_list):
    global duplicates
    duplicates = df[df['Original'].isin(train_list)]['Original'].tolist()
    df = df[~df['Original'].isin(train_list)]
    return df

# Update the dataframe by removing duplicates
df = check_duplicates_and_remove(summary, x_train)

In [ ]:
#remove spekaer 2 entries.
df = df[~df['Original'].isin(texts)]

In [ ]:
df

,Original,TrueTranslation,Translation,BLEU_Score,ROUGE-1,ROUGE-2,Exact_Match
9,تامر مرحبا,تامر مرحبا,تامر مرحبا,0.316228,1.0000,1.0000,True
13,وين موجود إنت اليوم,وين موجود إنت اليوم,وين موجود إنت اليوم,1.000000,1.0000,1.0000,True
15,بالد,باللد,بالد,0.000000,0.0000,0.0000,False
17,إنت سكان اللد ولا ولا حيفا,إنت سكان اللد ولا حيفا,أنت سكان اللد ولا حيفا,0.668740,0.8000,0.7500,False
19,والدار هادي عندك مش زي ما بنشوف على انستجرام و...,والدار هادي عندك مش زي ما بنشوف على انستجرام و...,والدار هادي عندك مش زي ما بنشوف على انستجرام و...,1.000000,1.0000,1.0000,True
...,...,...,...,...,...,...,...
8000,قديش سعر الواحد أسبوعية اي ينشر و ويشتغل على ا...,قديش سعر الواحد أسبوعية ينشر ويشتغل على المونتاج,قديش سعر الواحد أسبوعية ينشر ويشتغل على المونتاج,1.000000,1.0000,1.0000,True
8001,حتى لو منتج بسيط بس في شغل ويلاقي ضيوف أحكي,حتى لو منتج بسيط بس في شغل ويلاقي ضيوف أحكي,حتى لو منتج بسيط بس في شغل ويلاقي ضيوف أحكي,1.000000,1.0000,1.0000,True
8002,مع الضيوف وتوصل بودكاست ل ل46 حلقة هذا شيء مش,مع الضيوف وتوصل بودكاست ل ل46 حلقة هذا شيء مش,مع الضيوف وتوصل بودكاست ل46 حلقة هذا شيء مش,0.669048,0.9474,0.8235,False
8003,مفهوم ضمنهم أحمد,مفهوم ضمنهم أحمد,مفهوم ضمنا أحمد,0.135120,0.6667,0.0000,False


In [ ]:
df['BLEU_Score'].mean()

0.8017687511456836

In [ ]:
df['BLEU_Score'].median()

1.0

#### Reversed model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_path = "/content/drive/MyDrive/biqawi_alignment_model/reversed_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)


In [ ]:
texts = [str(text) for text in x_test]
true_translation = [str(text) for text in y_test]

In [ ]:
# Initialize lists to hold original texts and their translations
original_texts = []
translated_texts = []

# Set the batch size
batch_size = 16  # Adjust this based on your available memory

# Process the texts in batches
for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i + batch_size]

    # Tokenize the input texts
    inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=256)

    # Generate translations with max_new_tokens set to 256
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=256)

    # Decode the translations
    translations = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    # Append the batch results to the lists
    original_texts.extend(batch_texts)
    translated_texts.extend(translations)

# Save the original texts and translations in a DataFrame
pred_df = pd.DataFrame({'Original': original_texts, 'Translation': translated_texts})

In [ ]:
pred_df

,Original,Translation
0,كمان بس من ناحية مضمونية أنا دائما أنا إنسان إللي,اللي إنسان أنا دائما أنا مضمونية ناحية من بس كمان
1,هو دائما بالتواصل مع الطفل إللي بداخله أنا,أنا بداخله إللي الطفل مع بالتواصل دائما هو
2,ما ما ما بخليهم شيء كبار مابخليها الشيء كبار ف...,أعتقد طفولتنا في كبار هالشيء بخليهمش ما
3,إحنا كلنا بالأخص نحنا كعرب وبالأخص كذكور إللي ...,ما الطفل اللي كذكور وبالأخص كعرب نحنا بالأخص ك...
4,كان ما كان له وسع ما كان لإله وما,وما لإله كان ما وسع له كان ما
...,...,...
8006,لإلكم وشكرا على المتابعة إيه وبحب أذكرك معبد أ...,مني أشطر معبد أذكرك وبحب المتابعة على وشكرا لإلكم
8007,بالتذكير أنا بضل أنسى بالتذكير بالمشاركة المشا...,المشاركة مهمة كثير المشاركة بالمشاركة بالتذكير...
8008,بتعطي طاقة وبتعطي يعني طاقة عن جد دفعة كبيرة ل...,للاستمرارية كبيرة دفعة جد عن طاقة يعني وبتعطي ...
8009,أكثر وللبحث على ضيوف وعلى مواضيع بتهم بتهمكم ش...,لإلك شكرا بتهمكم بتهم مواضيع وعلى ضيوف على ولل...


In [ ]:
#reversing the machine translation - we expect the data to be in a similar orientation
pred_df['Translation'] = pred_df['Translation'].apply(reverse_sentence)
pred_df

,Original,Translation
0,كمان بس من ناحية مضمونية أنا دائما أنا إنسان إللي,كمان بس من ناحية مضمونية أنا دائما أنا إنسان اللي
1,هو دائما بالتواصل مع الطفل إللي بداخله أنا,هو دائما بالتواصل مع الطفل إللي بداخله أنا
2,ما ما ما بخليهم شيء كبار مابخليها الشيء كبار ف...,ما بخليهمش هالشيء كبار في طفولتنا أعتقد
3,إحنا كلنا بالأخص نحنا كعرب وبالأخص كذكور إللي ...,إحنا كلنا بالأخص نحنا كعرب وبالأخص كذكور اللي ...
4,كان ما كان له وسع ما كان لإله وما,ما كان له وسع ما كان لإله وما
...,...,...
8006,لإلكم وشكرا على المتابعة إيه وبحب أذكرك معبد أ...,لإلكم وشكرا على المتابعة وبحب أذكرك معبد أشطر مني
8007,بالتذكير أنا بضل أنسى بالتذكير بالمشاركة المشا...,بالتذكير أنا بضل أنسى بالتذكير بالمشاركة المشا...
8008,بتعطي طاقة وبتعطي يعني طاقة عن جد دفعة كبيرة ل...,بتعطي طاقة وبتعطي يعني طاقة عن جد دفعة كبيرة ل...
8009,أكثر وللبحث على ضيوف وعلى مواضيع بتهم بتهمكم ش...,أكثر وللبحث على ضيوف وعلى مواضيع بتهم بتهمكم ش...


In [ ]:
# Rename and drop irrelevant columns in og_df
true_translation = [str(text) for text in y_test]
og_df = pd.DataFrame({'Original': texts, 'TrueTranslation': true_translation})
og_df['TrueTranslation'] = og_df['TrueTranslation'].apply(remove_punctuation)
og_df


,Original,TrueTranslation
0,كمان بس من ناحية مضمونية أنا دائما أنا إنسان إللي,كمان بس من ناحية مضمونية أنا دائما أنا إنسان إللي
1,هو دائما بالتواصل مع الطفل إللي بداخله أنا,هو دائما بالتواصل مع الطفل إللي بداخله أنا
2,ما ما ما بخليهم شيء كبار مابخليها الشيء كبار ف...,ما بخليهوش يكبر ما بخليهوش يكبر في طفولتنا أعتقد
3,إحنا كلنا بالأخص نحنا كعرب وبالأخص كذكور إللي ...,إحنا كلنا بالأخص نحنا كعرب وبالأخص كذكور إللي ...
4,كان ما كان له وسع ما كان لإله وما,كان ما كان إله وسع ما كان لإله وسع وما
...,...,...
8006,لإلكم وشكرا على المتابعة إيه وبحب أذكرك معبد أ...,لإلكم وشكرا على المتابعة وبحب أذكرك معبد أشطر مني
8007,بالتذكير أنا بضل أنسى بالتذكير بالمشاركة المشا...,بالتذكير أنا بضل أنسى بالتذكير بالمشاركة المشا...
8008,بتعطي طاقة وبتعطي يعني طاقة عن جد دفعة كبيرة ل...,بتعطي طاقة وبتعطي يعني طاقة عن جد دفعة كبيرة ل...
8009,أكثر وللبحث على ضيوف وعلى مواضيع بتهم بتهمكم ش...,أكثر والبحث على ضيوف وعلى مواضيع بتهم بتهمكم ش...


In [ ]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.util import ngrams
from collections import Counter

# Merge the DataFrames on the 'Original' column
merged_df = pd.DataFrame({
    'Original': texts,
    'TrueTranslation': og_df['TrueTranslation'],
    'Translation': pred_df['Translation']
})

# Function to calculate BLEU score for each pair of translations
def calculate_bleu(reference, candidate):
    reference = reference.split()
    candidate = candidate.split()
    smoothing_function = SmoothingFunction().method1
    return sentence_bleu([reference], candidate, smoothing_function=smoothing_function)

# Function to calculate ROUGE scores
def calculate_rouge(reference, candidate):
    reference_tokens = reference.split()
    candidate_tokens = candidate.split()

    # Unigrams (ROUGE-1)
    reference_unigrams = set(ngrams(reference_tokens, 1))
    candidate_unigrams = set(ngrams(candidate_tokens, 1))
    common_unigrams = reference_unigrams.intersection(candidate_unigrams)
    precision_1 = len(common_unigrams) / (len(candidate_unigrams) + 1e-10)
    recall_1 = len(common_unigrams) / (len(reference_unigrams) + 1e-10)

    # Bigrams (ROUGE-2)
    reference_bigrams = set(ngrams(reference_tokens, 2))
    candidate_bigrams = set(ngrams(candidate_tokens, 2))
    common_bigrams = reference_bigrams.intersection(candidate_bigrams)
    precision_2 = len(common_bigrams) / (len(candidate_bigrams) + 1e-10)
    recall_2 = len(common_bigrams) / (len(reference_bigrams) + 1e-10)

    # F1-score for ROUGE-1 and ROUGE-2
    f1_1 = 2 * (precision_1 * recall_1) / (precision_1 + recall_1 + 1e-10)
    f1_2 = 2 * (precision_2 * recall_2) / (precision_2 + recall_2 + 1e-10)

    return round(f1_1, 4), round(f1_2, 4)

# Calculate BLEU and ROUGE scores, and exact matches
merged_df['BLEU_Score'] = merged_df.apply(lambda row: calculate_bleu(row['TrueTranslation'], row['Translation']), axis=1)
merged_df[['ROUGE-1', 'ROUGE-2']] = merged_df.apply(lambda row: calculate_rouge(row['TrueTranslation'], row['Translation']), axis=1, result_type='expand')
merged_df['Exact_Match'] = merged_df['TrueTranslation'] == merged_df['Translation']

# Calculate median and mean for BLEU_Score, ROUGE-1, and ROUGE-2
metrics_summary = {
    'BLEU_Score': {
        'mean': merged_df['BLEU_Score'].mean(),
        'median': merged_df['BLEU_Score'].median()
    },
    'ROUGE-1': {
        'mean': merged_df['ROUGE-1'].mean(),
        'median': merged_df['ROUGE-1'].median()
    },
    'ROUGE-2': {
        'mean': merged_df['ROUGE-2'].mean(),
        'median': merged_df['ROUGE-2'].median()
    }
}

# Display the updated summary and metrics summary
summary = merged_df[['Original', 'TrueTranslation', 'Translation', 'BLEU_Score', 'ROUGE-1', 'ROUGE-2', 'Exact_Match']]
summary


,Original,TrueTranslation,Translation,BLEU_Score,ROUGE-1,ROUGE-2,Exact_Match
0,كمان بس من ناحية مضمونية أنا دائما أنا إنسان إللي,كمان بس من ناحية مضمونية أنا دائما أنا إنسان إللي,كمان بس من ناحية مضمونية أنا دائما أنا إنسان اللي,0.880112,0.8889,0.8889,False
1,هو دائما بالتواصل مع الطفل إللي بداخله أنا,هو دائما بالتواصل مع الطفل إللي بداخله أنا,هو دائما بالتواصل مع الطفل إللي بداخله أنا,1.000000,1.0000,1.0000,True
2,ما ما ما بخليهم شيء كبار مابخليها الشيء كبار ف...,ما بخليهوش يكبر ما بخليهوش يكبر في طفولتنا أعتقد,ما بخليهمش هالشيء كبار في طفولتنا أعتقد,0.132014,0.6154,0.3333,False
3,إحنا كلنا بالأخص نحنا كعرب وبالأخص كذكور إللي ...,إحنا كلنا بالأخص نحنا كعرب وبالأخص كذكور إللي ...,إحنا كلنا بالأخص نحنا كعرب وبالأخص كذكور اللي ...,0.707107,0.9000,0.7778,False
4,كان ما كان له وسع ما كان لإله وما,كان ما كان إله وسع ما كان لإله وسع وما,ما كان له وسع ما كان لإله وما,0.332771,0.8333,0.4286,False
...,...,...,...,...,...,...,...
8006,لإلكم وشكرا على المتابعة إيه وبحب أذكرك معبد أ...,لإلكم وشكرا على المتابعة وبحب أذكرك معبد أشطر مني,لإلكم وشكرا على المتابعة وبحب أذكرك معبد أشطر مني,1.000000,1.0000,1.0000,True
8007,بالتذكير أنا بضل أنسى بالتذكير بالمشاركة المشا...,بالتذكير أنا بضل أنسى بالتذكير بالمشاركة المشا...,بالتذكير أنا بضل أنسى بالتذكير بالمشاركة المشا...,1.000000,1.0000,1.0000,True
8008,بتعطي طاقة وبتعطي يعني طاقة عن جد دفعة كبيرة ل...,بتعطي طاقة وبتعطي يعني طاقة عن جد دفعة كبيرة ل...,بتعطي طاقة وبتعطي يعني طاقة عن جد دفعة كبيرة ل...,1.000000,1.0000,1.0000,True
8009,أكثر وللبحث على ضيوف وعلى مواضيع بتهم بتهمكم ش...,أكثر والبحث على ضيوف وعلى مواضيع بتهم بتهمكم ش...,أكثر وللبحث على ضيوف وعلى مواضيع بتهم بتهمكم ش...,0.782542,0.9000,0.7778,False


In [ ]:
metrics_summary

{'BLEU_Score': {'mean': 0.6849387911582617, 'median': 0.7259795291154771},
 'ROUGE-1': {'mean': 0.8911772063412808, 'median': 1.0},
 'ROUGE-2': {'mean': 0.7749452128323555, 'median': 0.8889}}

In [ ]:
# Optionally, save the DataFrame to a CSV file
summary.to_csv("/content/drive/MyDrive/biqawi_alignment_model/translations_reverse.csv", index=False)

In [ ]:
filtered_summary = summary[(summary['BLEU_Score'] != 1) & (summary['Exact_Match'] == True)]
filtered_summary

,Original,TrueTranslation,Translation,BLEU_Score,ROUGE-1,ROUGE-2,Exact_Match
9,تامر مرحبا,تامر مرحبا,تامر مرحبا,0.316228,1.0,1.0,True
10,شو الوضع,شو الوضع,شو الوضع,0.316228,1.0,1.0,True
12,تمام الحمد لله,تمام الحمد لله,تمام الحمد لله,0.562341,1.0,1.0,True
20,وهيك,وهيك,وهيك,0.177828,1.0,0.0,True
30,قديه أعمارهم,قديه أعمارهم,قديه أعمارهم,0.316228,1.0,1.0,True
...,...,...,...,...,...,...,...
7986,يعني من قلب,يعني من قلب,يعني من قلب,0.562341,1.0,1.0,True
7996,الخميس,الخميس,الخميس,0.177828,1.0,0.0,True
8003,مفهوم ضمنهم أحمد,مفهوم ضمنهم أحمد,مفهوم ضمنهم أحمد,0.562341,1.0,1.0,True
8004,آه وبالتوفيق بالبرنامج,وبالتوفيق بالبرنامج,وبالتوفيق بالبرنامج,0.316228,1.0,1.0,True


In [ ]:
'''
This script performs several tasks related to translation evaluation and comparison.
1. **Data Merging**:
   - It merges two DataFrames (`og_df` and `df`) based on the common column 'Original'.
   - The resulting DataFrame is stored in `merged_df`.

2. **BLEU Score Calculation**:
   - The `calculate_bleu` function computes the BLEU score between a reference translation and a candidate translation.
   - Both the reference and candidate translations are tokenized using `.split()`.
   - The BLEU score is calculated with smoothing using `SmoothingFunction().method1`.
   - The scores are added to the `merged_df` DataFrame.

3. **Exact Match Check**:
   - An 'Exact_Match' column is added to `merged_df`, indicating whether the candidate translation exactly matches the true reference translation.

4. **Tokenization for Comparison**:
   - The true and model translations are tokenized into lists of words (`TrueTokens` and `ModelTokens`) for further comparison.

5. **Identifying Mismatches**:
   - The `collect_mismatches` function compares corresponding tokens in the true and model translations.
   - If a token mismatch occurs, it's recorded in the 'Mismatches' column of `merged_df`.

6. **Counting and Displaying Common Mistakes**:
   - All mismatches are flattened into a single list (`all_mismatches`).
   - The `Counter` is used to count occurrences of each mismatch.
   - The most common mistakes are printed to the console.

7. **Optional: Saving Mistakes to CSV**:
   - If desired, the common mistakes can be saved to a CSV file in a DataFrame called `mistakes_df`.
'''
# Import necessary libraries
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from collections import Counter

merged_df = pd.DataFrame({
    'Original': texts,
    'TrueTranslation': og_df['TrueTranslation'],
    'Translation': pred_df['Translation']
})

# Function to calculate BLEU score for each pair of translations
def calculate_bleu(reference, candidate):
    reference = reference.split()  # Tokenize the reference translation
    candidate = candidate.split()  # Tokenize the candidate translation
    smoothing_function = SmoothingFunction().method1
    return sentence_bleu([reference], candidate, smoothing_function=smoothing_function)

# Calculate BLEU scores and exact matches
merged_df['BLEU_Score'] = merged_df.apply(lambda row: calculate_bleu(row['TrueTranslation'], row['Translation']), axis=1)
merged_df['Exact_Match'] = merged_df['TrueTranslation'] == merged_df['Translation']

# Tokenize the translations for comparison
merged_df['TrueTokens'] = merged_df['TrueTranslation'].apply(lambda x: x.split())  # Tokenize true translations
merged_df['ModelTokens'] = merged_df['Translation'].apply(lambda x: x.split())  # Tokenize model translations

# Identify and collect mismatches
def collect_mismatches(true_tokens, model_tokens):
    mismatches = []
    for true, model in zip(true_tokens, model_tokens):
        if true != model:
            mismatches.append((true, model))
    return mismatches

merged_df['Mismatches'] = merged_df.apply(lambda row: collect_mismatches(row['TrueTokens'], row['ModelTokens']), axis=1)

# Flatten the list of mismatches and count occurrences
all_mismatches = [mismatch for sublist in merged_df['Mismatches'] for mismatch in sublist]
mismatch_counter = Counter(all_mismatches)

# Display the most common mistakes
common_mistakes = mismatch_counter.most_common()
# for mistake, count in common_mistakes:
#     print(f"Mistake: {mistake}, Count: {count}")

# Optionally, save the common mistakes to a CSV file
mistakes_df = pd.DataFrame(common_mistakes, columns=['Mistake', 'Count'])
mistakes_df

,Mistake,Count
0,"(هلق, هلأ)",35
1,"(نحنا, إحنا)",28
2,"(إللي, اللي)",21
3,"(فيش, فش)",14
4,"(إللي, هو)",10
...,...,...
11279,"(ل46, حلقة)",1
11280,"(حلقة, هذا)",1
11281,"(هذا, شيء)",1
11282,"(شيء, مش)",1
